### RunnablePassthrough, RunnableLambda, RunnableParallel

##### Boilerplate Code

In [ ]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)


### Callable - plain Python function/lambda/class with __call__, can be executed with ()

In [ ]:
# Callable
def add_one(x):
    return x + 1

add_two = lambda x: x + 2

print(add_one.__call__(1))
print(add_two.__call__(1))


In [18]:
# Runnable (wrapping callable)
from langchain.schema.runnable import RunnableLambda
runnable = RunnableLambda(add_two)

print(runnable.invoke(1))

3


### RunnablePassthrough - just echoes the input unchanged

In [19]:
from langchain_core.runnables import RunnablePassthrough

# Simple example - just passes data through
chain = RunnablePassthrough()
result = chain.invoke("Hello World")
print(result)


Hello World


In [22]:
# Adding extra data while keeping original
chain = RunnablePassthrough.assign(
    length=lambda x: len(x["text"]),
    upper=lambda x: x["text"].upper()
)

result = chain.invoke({"text": "Python"})
print(result)

{'text': 'Python', 'length': 6, 'upper': 'PYTHON'}


### RunnableLambda - wraps a Python function as a runnable.

In [23]:
from langchain_core.runnables import RunnableLambda

# Simple function
def make_uppercase(text):
    return text.upper()

# Turn it into a Runnable
uppercase_step = RunnableLambda(make_uppercase)

result = uppercase_step.invoke("hello world")
print(result)


HELLO WORLD


In [24]:
# Chain multiple functions
def add_excitement(text):
    return text + "!!!"

chain = (
    RunnableLambda(make_uppercase) |  # First: make uppercase
    RunnableLambda(add_excitement)    # Then: add exclamation
)

result = chain.invoke("hello world")
print(result)

HELLO WORLD!!!


### RunnableParallel - runs multiple runnables at the same time and merges results.

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

# Define simple functions
def count_words(text):
    return len(text.split())

def count_chars(text):
    return len(text)

def make_uppercase(text):
    return text.upper()

# Run all functions at the same time on the same input
parallel = RunnableParallel(
    words=RunnableLambda(count_words),
    chars=RunnableLambda(count_chars),
    upper=RunnableLambda(make_uppercase)
)

input_text = "I love Python"
result = parallel.invoke(input_text)
print(result)


### Merging all - RunnablePassthrough, RunnableLambda, RunnableParallel in single code

In [27]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

# Simple functions
def count_words(text):
    return len(text.split())

def get_first_word(text):
    return text.split()[0]

# Build a simple chain
chain = (
    RunnablePassthrough.assign(
        analysis=RunnableParallel(
            word_count=RunnableLambda(lambda d: count_words(d["text"])),
            first_word=RunnableLambda(lambda d: get_first_word(d["text"]))
        )
    )
)

# Test it
result = chain.invoke({"text": "Python programming is fun"})
print(result)



ValueError: The input to RunnablePassthrough.assign() must be a dict.